In [8]:
from pydantic import BaseModel, Field
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [17]:
class CheckResult(BaseModel):
    task_name: str = Field(..., description="Name of the task being checked. Keep exactly the same as the task name in the request. Normally it is bullet point")
    is_passed: bool = Field(..., description="Whether the task was completed successfully")
    description: str = Field(..., description="If the task was passed, a description is explain where it contains the result. If not, a description of the error")

class TaskCheckResponses(BaseModel):
    results: list[CheckResult]

In [18]:
task = '''You have decided to leave your current job and look for a new one.

Write a letter to a friend. In your letter

explain why you want to leave your current job
describe what kind of job you are looking for
ask for some help or advice'''
response = '''Dear Luca,

I am writing to let you know about my current job situation, hoping that you can give me some advice. I have decided to leave my current position at the Swaneset Company. I have worked there for several years now, and I think I have advanced as far as possible in this company. I would like to work at a larger company where there are more opportunities for advancement.

As you know, my most recent position at Swaneset has been as a software engineer in the IT Department. I have learned a great deal about this type of work and am very interested in it. I am hoping to find a similar sort of position at a larger company. I do not mind starting at a new company at the same level as I am working at now, as long as I feel there are good opportunities to move up before too long.

Since you have a lot of experience in this field, I was hoping that you could help me identify some companies that might be a good fit for me. I would also appreciate some help with my resume, if you do not mind my asking. I hope to hear from you soon.

Your friend,
John'''

In [19]:
client = OpenAI()
completion = client.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Your duty is to check if the user has completed the tasks requested in their writing."},
        {"role": "user", "content": f"Given tasks: {task} \nResponse: {response}\nPlease check if the tasks are completed and provide a detailed report."},
    ],
    response_format=TaskCheckResponses,
)

result = completion.choices[0].message.parsed

In [20]:
result.results

[CheckResult(task_name='Write a letter to a friend.', is_passed=True, description="The letter is addressed to a friend, Luca, and discusses the user's job situation."),
 CheckResult(task_name='Explain why you want to leave your current job', is_passed=True, description='The letter states that the user feels they have advanced as far as possible in their current job at Swaneset Company.'),
 CheckResult(task_name='Describe what kind of job you are looking for', is_passed=True, description='The user describes wanting a similar position at a larger company with more opportunities for advancement.'),
 CheckResult(task_name='Ask for some help or advice', is_passed=True, description='The user explicitly asks Luca for help identifying suitable companies and assistance with their resume.')]